In [76]:
%cd /home/nikita91/Documents/VScodeProjects/RNN-TimeSeries-coursework/convformer

/home/nikita91/Documents/VScodeProjects/RNN-TimeSeries-coursework/convformer


In [77]:
import os
# import argparse
import torch

# Make sure the repo root (containing data_provider, exp, models, utils) is in PYTHONPATH,
# or adjust the path below if needed:
import sys
sys.path.append(os.getcwd())  # Adjust if your working directory is not the repo root

# Import the experiment class
from exp.exp_main import Exp_Main


In [78]:
# Define a simple argument namespace (similar to argparse.Namespace)
class Args:
    def __init__(self):
        # Model selection: choose one of ['Autoformer', 'Informer', 'Transformer', 'Reformer']
        self.model = 'Informer'

        # Device settings
        self.use_gpu = True  # Set to True if CUDA is available and you want to use GPU
        self.gpu = 0
        self.devices = '0,1,2,3'
        self.use_multi_gpu = False

        # Data settings (ensure these paths match your dataset locations)
        self.root_path = './data/raw/ETT-small/'          # Root directory for dataset
        self.data = 'ETTh1'                 # dataset type
        self.data_path = 'ETTh1.csv'        # Example CSV file
        self.features = 'M'                 # 'M' for multivariate input, 'S' for single
        self.target = 'OT'                  # Column name to predict
        self.freq = 'h'                     # Time frequency of your data (e.g., 'h' for hourly)
        self.checkpoints = './checkpoints/' # Directory to save checkpoints

        # Sequence lengths
        self.seq_len = 96     # Input sequence length
        self.label_len = 48   # Length for the decoder input
        self.pred_len = 48    # Prediction horizon

        # Model hyperparameters (basic defaults)
        self.enc_in = 7       # Number of input features
        self.dec_in = 7       # Number of features fed to decoder
        self.c_out = 7        # Number of output features ???
        self.d_model = 512    # Dimension of model
        self.n_heads = 8      # Number of attention heads
        self.e_layers = 2     # Number of encoder layers
        self.d_layers = 1     # Number of decoder layers
        self.d_ff = 2048      # Dimension of feed-forward network
        self.moving_avg = 25  # Window size for moving average decomposition
        self.factor = 3       # Attention factor for ProbSparse
        # self.factor = 5       # Attention factor for ProbSparse
        self.distil = True    # Use distillation in encoder (True/False)
        self.dropout = 0.05  # Dropout probability
        self.embed = 'timeF'  # Time feature embedding method
        self.activation = 'gelu'  # Activation function
        self.output_attention = False  # Whether to output attention weights

        # Optimization settings
        self.loss = 'mse'     # Loss type (e.g., 'mse')
        self.lradj = 'type1'  # Learning rate adjustment type
        self.learning_rate = 0.0001
        self.batch_size = 32
        self.train_epochs = 10  # For demo, keep epochs small
        self.patience = 3      # Early stopping patience
        self.num_workers = 10   # Parallel data loading

        # Mixed precision and multi-GPU (for simplicity, keep disabled)
        self.use_amp = False

        # Experiment description
        self.des = 'simple_demo'

        # Token for prediction-only mode (unused here)
        self.do_predict = False

    def __str__(self):
        lines = []
        for name, val in sorted(self.__dict__.items()):
            lines.append(f"{name:15s}: {val}")
        return "\n".join(lines)

# Instantiate args
args = Args()
print(f'cuda is available: {torch.cuda.is_available()}')
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print(f'cuda is being used: {args.use_gpu}')

# print(args)


cuda is available: True
cuda is being used: True


In [79]:
# Create an experiment instance
exp = Exp_Main(args)

# Define a simple setting name (used for saving checkpoints/results)
setting = 'informer_48_default'

# Run training (will train for args.train_epochs epochs)
print('Starting training...')
exp.train(setting)
print('Training completed.')


Use GPU: cuda:0
Starting training...
train 8497
val 2833
test 2833
	iters: 100, epoch: 1 | loss: 0.4592566
	speed: 0.0887s/iter; left time: 226.1979s
	iters: 200, epoch: 1 | loss: 0.3673691
	speed: 0.0862s/iter; left time: 211.2752s
Epoch: 1 cost time: 23.194034576416016
Epoch: 1, Steps: 265 | Train Loss: 0.4234568 Vali Loss: 0.7714586 Test Loss: 0.5975754
Validation loss decreased (inf --> 0.771459).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.3109574
	speed: 0.2040s/iter; left time: 466.3592s
	iters: 200, epoch: 2 | loss: 0.2606356
	speed: 0.0864s/iter; left time: 188.9185s
Epoch: 2 cost time: 23.22842812538147
Epoch: 2, Steps: 265 | Train Loss: 0.2888134 Vali Loss: 0.7752330 Test Loss: 0.5804874
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.2595224
	speed: 0.2041s/iter; left time: 412.4726s
	iters: 200, epoch: 3 | loss: 0.2397701
	speed: 0.0871s/iter; left time: 167.3098s
Epoch: 3 cost time: 2

In [80]:
# Run testing to evaluate on the test set (loads the best checkpoint)
print('Starting testing...')
exp.test(setting, test=1)
print('Testing completed.')


Starting testing...
test 2833
loading model
test shape: (2833, 48, 7) (2833, 48, 7)
test shape: (2833, 48, 7) (2833, 48, 7)
mse:0.5975908041000366, mae:0.572128415107727
Testing completed.


## Prediction on New Data
If you have a separate `pred` split configured, you can run:
```python
exp.predict(setting, load=True)
```